# Case 2: Siemens AI-Driven Sales Forecasting

## Overview
This case study involves building a monthly sales forecasting model using real sales data from Siemens’ Smart Infrastructure Division in Germany. The objective is to apply machine learning techniques to predict future sales based on historical data and macro-economic indicators.

## Business Problem
- Manual sales forecasting is time-consuming and relies on human judgment.  
- Data is scattered across multiple sources, making it difficult to derive insights.  
- Inaccurate forecasts lead to financial losses, such as inefficient inventory management and unsatisfied customers.

## Objective
- Develop an AI-driven predictive model to automate the forecasting process.  
- Evaluate the model using Root Mean Squared Error (RMSE).  
- Submit predictions for May 2022 - February 2023 in a structured CSV format.


---

**This notebook was developed by:**
- João Venichand - 20211644  
- Gonçalo Custódio - 20211643  
- Diogo Correia - 20211586  
- Duarte Emanuel - 20240564


### 1. Import the Libraries


In [ ]:
import calendar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore, boxcox
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from prophet import Prophet
from sklearn.model_selection import GridSearchCV

#warnings.simplefilter(action='ignore', category=FutureWarning)

### 2. Load Datasets

Import Sales Data

In [ ]:
sales_data = pd.read_csv("sales_data.csv")
sales_data.head(5)

Import Market Data

In [ ]:
market_data = pd.read_excel("market_data.xlsx", header=None)
market_data.head(5)

### 3. Sales Dataset Analysis

Check for missing values

In [ ]:
missing_values = sales_data["Sales_EUR"].isna().sum()
missing_values

Count occurrences of 0 and non-zero Sales

In [ ]:
zero_count = (sales_data["Sales_EUR"] == 0).sum()
non_zero_count = (sales_data["Sales_EUR"] != 0).sum()

print(f"Zero Sales Count: {zero_count}")
print(f"Non-Zero Sales Count: {non_zero_count}")

Sales Distribution by Product (excluding 0 sales values)

In [ ]:
"""filtered_sales = sales_data[sales_data["Sales_EUR"] != 0]
unique_products = filtered_sales["Mapped_GCK"].unique()

for product in unique_products:
    product_data = filtered_sales[filtered_sales["Mapped_GCK"] == product]["Sales_EUR"]
    
    plt.figure(figsize=(8, 5))
    plt.boxplot(product_data, vert=False)
    plt.title(f"Sales Distribution for Product {product} (excluding 0 sales)")
    plt.xlabel("Sales_EUR")
    plt.grid(True)
    plt.show()"""

Check for Duplicate Dates and Their Frequencies

In [ ]:
sales_data["DATE"] = pd.to_datetime(sales_data["DATE"], format="%d.%m.%Y")
duplicate_dates = sales_data["DATE"].duplicated().sum()
date_counts = sales_data["DATE"].value_counts().sort_index()

print(f"Number of duplicate dates: {duplicate_dates}")
print("\nFirst 10 occurrences of dates:")
print(date_counts.head(10))

Quantity of Different Products

In [ ]:
unique_values = sales_data["Mapped_GCK"].unique()

num_unique_values = len(unique_values)

print(f"Number of unique values in Mapped_GCK: {num_unique_values}")
print("Unique values:")
print(unique_values)

Distribution of Values by Product

In [ ]:
mapped_gck_counts = sales_data["Mapped_GCK"].value_counts()
print("Count of each unique value in Mapped_GCK:")
print(mapped_gck_counts)

total_rows = len(sales_data)
print(f"\nTotal number of rows in the dataset: {total_rows}")

Check for Duplicates 

In [ ]:
duplicates_by_gck = sales_data.groupby("Mapped_GCK")["DATE"].apply(lambda x: x.duplicated().sum())

print("Duplicate DATEs per Mapped_GCK:")
print(duplicates_by_gck)

Analyze Average Sales by Weekday, Month, and Year

In [ ]:
sales_data["Weekday"] = sales_data["DATE"].dt.day_name()
sales_data["Year"] = sales_data["DATE"].dt.year
sales_data["Month"] = sales_data["DATE"].dt.month
weekday_sales = sales_data.groupby("Weekday")["Sales_EUR"].mean().sort_values()
month_sales = sales_data.groupby("Month")["Sales_EUR"].mean().sort_values()
year_sales = sales_data.groupby("Year")["Sales_EUR"].mean().sort_values()

In [ ]:
print(weekday_sales)

In [ ]:
print(month_sales)

In [ ]:
print(year_sales)

Visualize Average Sales per Product

In [ ]:
mapped_gck_means = sales_data.groupby("Mapped_GCK")["Sales_EUR"].mean().sort_values()

plt.figure(figsize=(12, 6))
plt.bar(mapped_gck_means.index, mapped_gck_means.values)
plt.xticks(rotation=90)
plt.xlabel("Mapped_GCK (Product)")
plt.ylabel("Average Sales")
plt.title("Mean Sales per Mapped_GCK")
plt.grid(True)
plt.show()

Mean Sales per Product

In [ ]:
mapped_gck_means = sales_data.groupby("Mapped_GCK")["Sales_EUR"].mean().sort_values()

print("Mean Sales per Product:")
print(mapped_gck_means)

Check the Sales_Data

In [ ]:
sales_data.head(5)

### 4. Market Data Analysis

 Preview first rows of Market Data

In [ ]:
market_data.head(5)

Clean and Reformat Market Data Header

In [ ]:
market_data = market_data.drop(index=2).reset_index(drop=True)

new_columns = [
    f"{market_data.iloc[0, i]} - {market_data.iloc[1, i]}" if pd.notna(market_data.iloc[0, i]) 
    else market_data.iloc[1, i] 
    for i in range(market_data.shape[1])
]

market_data.columns = new_columns
market_data = market_data.iloc[2:].reset_index(drop=True)
market_data = market_data.rename(columns={market_data.columns[0]: "Date"})

Extract Year and Month from Date Column

In [ ]:
market_data["Date"] = market_data["Date"].str.strip()
market_data["Year"] = market_data["Date"].str[:4].astype(int)
market_data["Month"] = market_data["Date"].str[5:].astype(int)

Validate Year and Month Extraction

In [ ]:
year_counts = market_data["Year"].value_counts().sort_index()
month_counts = market_data["Month"].value_counts().sort_index()

print("Unique Years and Counts:")
print(year_counts)

print("\nUnique Months and Counts:")
print(month_counts)

### 5. Feature Selection

#### Selection of Key Macroeconomic Variables

Based on our analysis and discretion, we believe that the following variables are the most important for predicting Siemens' monthly sales . These variables were selected to balance **demand-side factors, cost influences, and production constraints**, while following the expert advice of using **fewer but high-value features**.

#### **Production and Shipment Indices**

- **China - Production Index Machinery & Electricals**  and **United States - Production Index Machinery & Electricals**  
   Measures industrial production in Siemens’ key client countries. Higher values may reflect increased domestic demand for Siemens-related goods.

- **China - Shipments Index Machinery & Electricals** and **United States - Shipments Index Machinery & Electricals**  
   Tracks shipment volumes in the machinery and electrical sector. Growth suggests stronger export performance and external demand.

- **Germany - Production Index Machinery & Electricals** and **Germany - Shipments Index Machinery & Electricals**  
   Reflect local industrial and distribution activity. Especially relevant as the sales data comes from a German branch.


#### **Producer Prices (Cost of Siemens' Industrial Goods)**

- **Producer Prices - United States: Electrical Equipment**  
   Reflects how costly electrical equipment is for Siemens’ U.S.-based clients. Higher prices may reduce purchasing power; lower prices could support demand.

- **Producer Prices - China: Electrical Equipment**  
   Indicates equipment costs in China, potentially affecting Siemens’ production expenses and pricing strategies.

- **Producer Prices - Germany: Electrical Equipment**  
   Relevant to local operations. Fluctuations in German producer prices may influence both profitability and sales performance, since data is from a German branch.


#### **Raw Material and Energy Costs**
- **World: Price of Base Metals**  
   Siemens’ industrial products depend on metals like steel and aluminum. Higher prices increase production costs.

- **World: Price of Energy**  
   Rising energy costs elevate manufacturing expenses, potentially influencing Siemens’ pricing strategies.

- **World: Price of Crude Oil, Average**  
   Affects transportation and supply chain costs. Significant increases may reduce industrial investment and weigh on Siemens’ sales.

- **World: Price of Copper**  
   A key input in electrical equipment manufacturing.

These selected variables cover both **demand-side** factors (production and shipments), **cost structures** (producer prices), and **macro-level influences** (raw material and energy prices). This approach ensures a balanced, minimal, and high-impact feature selection.


Correlation Analysis of Selected Macroeconomic Features

In [ ]:
macro_features = [
    "China - Production Index Machinery & Electricals",
    "United States - Production Index Machinery & Electricals",
    "Germany - Production Index Machinery & Electricals",
    "China - Shipments Index Machinery & Electricals",
    "United States - Shipments Index Machinery & Electricals",
    "Germany - Shipments Index Machinery & Electricals",
    "Producer Prices - United States: Electrical equipment",
    "Producer Prices - China: Electrical equipment",
    "Producer Prices - Germany: Electrical equipment",
    "World: Price of Base Metals",
    "World: Price of Energy",
    "World: Price of Crude oil, average",
    "World: Price of Copper",
    "World: Price of Metals  & Minerals",
    "World: Price of Natural gas index"
]

correlation_matrix = market_data[macro_features].corr()

plt.figure(figsize=(10, 7))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Selected Macroeconomic Features")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

### Feature Selection Strategy

#### **Consolidating Redundant Features**
##### **China - Production Index Machinery & Electricals**
This variable exhibits **very high correlation** with multiple other features:
- **1.00** with `China - Shipments Index Machinery & Electricals`
- **0.74** with `United States - Shipments Index Machinery & Electricals`
- **0.73** with `Germany - Shipments Index Machinery & Electricals`
- **0.89** with `Producer Prices - United States: Electrical Equipment`
- **0.95** with `Producer Prices - Germany: Electrical Equipment`

Given this extreme correlation, we will **retain "China - Production Index Machinery & Electricals" as a representative variable**, allowing it to encapsulate the information from these **six** highly correlated features. 


##### **Aggregating World Price Features**
- **World: Price of Base Metals**  
- **World: Price of Energy**  
- **World: Price of Crude Oil, Average**  
- **World: Price of Copper**  

Our decision to merge these indicators was based on their **high correlation** and to to preserve shared economic influence while reducing redundancy.


#### **Removing Weakly Correlated Features**
The remaining three features:
- **Producer Prices - China: Electrical Equipment**
- **United States - Production Index Machinery & Electricals**
- **Germany - Production Index Machinery & Electricals**

show some moderate correlations. Their effects will already be reflected in the merged features, so we **exclude them** to keep the model clean and efficient.



Aggregated the World Price Features

In [ ]:
market_data["Key_Materials_Price_Index"] = (
    market_data["World: Price of Base Metals"] +
    market_data["World: Price of Energy"] +
    market_data["World: Price of Crude oil, average"] +
    market_data["World: Price of Copper"] +
    market_data["World: Price of Metals  & Minerals"] +
    market_data["World: Price of Natural gas index"] +
    market_data["Producer Prices - Germany: Electrical equipment"]
) / 7

### 6. Remove Outliers

Monthly Sales Aggregation by Product Group

In [ ]:
monthly_sales_data = sales_data.groupby(["Mapped_GCK", "Year", "Month"])["Sales_EUR"].sum().reset_index()

print(monthly_sales_data)

Distribution of Monthly Sales by Product

In [ ]:
"""unique_products = monthly_sales_data["Mapped_GCK"].unique()

for product in unique_products:
    plt.figure(figsize=(8, 4))
    plt.hist(
        monthly_sales_data[monthly_sales_data["Mapped_GCK"] == product]["Sales_EUR"],
        bins=50, edgecolor="black"
    )
    plt.title(f"Distribution of Sales_EUR for Product {product}")
    plt.xlabel("Sales_EUR")
    plt.ylabel("Frequency")
    plt.show()"""

Check the Negative Values in Sales_EUR

In [ ]:
negative_values = monthly_sales_data[monthly_sales_data["Sales_EUR"] < 0]
print(negative_values)

Set Negative Sales Values to 0

In [ ]:
monthly_sales_data["Sales_EUR"] = monthly_sales_data["Sales_EUR"].clip(lower=0)

Apply Log Transformation

In [ ]:
epsilon = abs(monthly_sales_data["Sales_EUR"].min()) + 1 
monthly_sales_data["Sales_EUR_Log"] = np.log1p(monthly_sales_data["Sales_EUR"] + epsilon)
monthly_sales_data.head()

In [ ]:
plt.hist(monthly_sales_data["Sales_EUR_Log"], bins=30, edgecolor="black")
plt.title("Distribution of Sales_EUR after Log Transform")
plt.xlabel("Sales_EUR_Log")
plt.ylabel("Frequency")
plt.show()

Describe the Sales_EUR_Log

In [ ]:
print(monthly_sales_data["Sales_EUR_Log"].describe())

Box-Cox Transformation

In [ ]:
monthly_sales_data["Sales_EUR_Shifted"] = monthly_sales_data["Sales_EUR"] + abs(monthly_sales_data["Sales_EUR"].min()) + 1
monthly_sales_data["Sales_EUR_BoxCox"], lambda_ = boxcox(monthly_sales_data["Sales_EUR_Shifted"])

print(f"Optimal Box-Cox Lambda: {lambda_}")

To reduce the impact of outliers and stabilize variance in Sales_EUR, we tested a Box-Cox transformation. The resulting lambda was 0.066, which is very close to zero and this confirms that a log transformation is already appropriate for our data, as Box-Cox behaves like log(x) when λ ≈ 0.

We therefore **kept the log transformation** as our chosen method for normalizing sales values.

Monthly Sales Trend for Product

In [ ]:
"""monthly_sales_data["Date"] = pd.to_datetime(monthly_sales_data["Year"].astype(str) + "-" + monthly_sales_data["Month"].astype(str) + "-01")
unique_products = monthly_sales_data["Mapped_GCK"].unique()

for product in unique_products:
    product_data = monthly_sales_data[monthly_sales_data["Mapped_GCK"] == product]
    
    plt.figure(figsize=(10, 5))
    plt.plot(product_data["Date"], product_data["Sales_EUR"], marker="o", linestyle="-")
    plt.title(f"Monthly Sales Trend for Product {product}")
    plt.xlabel("Month")
    plt.ylabel("Total Sales (EUR)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

monthly_sales_data = monthly_sales_data.drop(columns=["Date"])"""

### 7. Building the Final Dataset

Merging Data to Create Final Dataset

In [ ]:
print(market_data.columns.tolist())

In [ ]:
market_selected = market_data[
    ["Year", "Month", "Key_Materials_Price_Index",
     "China - Production Index Machinery & Electricals",
     "United States - Production Index Machinery & Electricals",
     "Germany - Production Index Machinery & Electricals",
     "Europe - Production Index Machinery & Electricals",
     "China - Shipments Index Machinery & Electricals",
     "United States - Shipments Index Machinery & Electricals",
     "Germany - Shipments Index Machinery & Electricals",
     "Producer Prices - Germany: Electrical equipment",
     "Producer Prices - United States: Electrical equipment",
     "Producer Prices - China: Electrical equipment",
     "United States: EUR in LCU"]
]
merged_data = monthly_sales_data.merge(market_selected, on=["Year", "Month"], how="left")

print(merged_data.head()) 

Renaming Columns for Clarity

In [ ]:
merged_data.rename(columns={
    "China - Production Index Machinery & Electricals": "Global_Industrial_Activity_Index",
    "World: Price of Metals & Minerals": "Metals_Minerals_Price",
    "World: Price of Natural Gas": "Natural_Gas_Price",
    "World: Producer Prices": "Producer_Price_Index"
}, inplace=True)

Converting Product Variable to Integer Type

In [ ]:
merged_data["Mapped_GCK"] = merged_data["Mapped_GCK"].str.replace("#", "").str.strip()
merged_data["Mapped_GCK"] = merged_data["Mapped_GCK"].astype(int)

In [ ]:
merged_data["Quarter"] = merged_data["Month"].apply(lambda x: (x - 1) // 3 + 1)
merged_data["Is_Holiday_Month"] = merged_data["Month"].apply(lambda x: 1 if x == 12 else 0)
merged_data["Sales_Lag_1M"] = merged_data.groupby("Mapped_GCK")["Sales_EUR"].shift(1)
merged_data["Sales_Lag_12M"] = merged_data.groupby("Mapped_GCK")["Sales_EUR"].shift(12)

In [ ]:
merged_data.fillna({"Sales_Lag_1M": merged_data["Sales_EUR"].mean(),
                    "Sales_Lag_12M": merged_data["Sales_EUR"].mean()}, inplace=True)

In [ ]:
print(merged_data.columns.tolist())

In [ ]:
correlation_matrix = merged_data[
    ["Key_Materials_Price_Index", "Sales_EUR",
     "Global_Industrial_Activity_Index",
     "United States - Production Index Machinery & Electricals",
     "Germany - Production Index Machinery & Electricals",
     "Europe - Production Index Machinery & Electricals",
     "Producer Prices - Germany: Electrical equipment",
     "Producer Prices - United States: Electrical equipment",
     "United States: EUR in LCU"]
].corr()

plt.figure(figsize=(10, 7))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap: Key Macroeconomic Features & Sales")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

Removing Unused Transformation Columns

In [ ]:
merged_data = merged_data.drop(columns=[
    "Sales_EUR_BoxCox",
    "Sales_EUR_Shifted",
    "Key_Materials_Price_Index",
    "Producer Prices - Germany: Electrical equipment",
    "United States: EUR in LCU"
])

In [ ]:
merged_data

Copy with "YearMonth" 

In [ ]:
merged_data_2 = merged_data.copy()
merged_data_2["YearMonth"] = (
    merged_data_2["Year"].astype(str) + "-" + 
    merged_data_2["Month"].astype(str).str.zfill(2)
)

merged_data_2 = merged_data_2.drop(columns=["Year", "Month"])
merged_data_2

# 8. Modelling

Time-Based Train-Test Split

In [ ]:
merged_data_2 = merged_data_2.sort_values(by=["YearMonth"])

split_point = int(len(merged_data_2) * 0.8)
train_data, test_data = merged_data_2.iloc[:split_point], merged_data_2.iloc[split_point:]

target = "Sales_EUR_Log"
excluded_features = ["YearMonth", "Sales_EUR", "Sales_EUR_Log"]
features = [col for col in merged_data_2.columns if col not in excluded_features]

X_train, X_test = train_data[features], test_data[features]
y_train, y_test = train_data[target], test_data[target]

print(f"Train set size: {len(train_data)} rows")
print(f"Test set size: {len(test_data)} rows")
print("Missing values in train set:", X_train.isnull().sum().sum())
print("Missing values in test set:", X_test.isnull().sum().sum())

In [ ]:
X_test.loc[:, "United States - Shipments Index Machinery & Electricals"] = X_test["United States - Shipments Index Machinery & Electricals"].fillna(X_train["United States - Shipments Index Machinery & Electricals"].mean())

Calculating Acceptable RMSE Threshold per Product

In [ ]:
ACCEPTABLE_PERCENTAGE = 20  
rmse_thresholds = []

for product_id in train_data["Mapped_GCK"].unique():
    median_sales = train_data[train_data["Mapped_GCK"] == product_id]["Sales_EUR"].median()
    threshold = (ACCEPTABLE_PERCENTAGE / 100) * median_sales
    rmse_thresholds.append([product_id, int(threshold)])

rmse_threshold_df = pd.DataFrame(rmse_thresholds, columns=["Mapped_GCK", "RMSE_Threshold"])
rmse_threshold_df

# Prophet

In [ ]:
forecast_results = []

for product_id in merged_data_2["Mapped_GCK"].unique():
    product_data = merged_data_2[merged_data_2["Mapped_GCK"] == product_id].copy()

    prophet_df = product_data[['YearMonth', 'Sales_EUR_Log']].rename(columns={'YearMonth': 'ds', 'Sales_EUR_Log': 'y'})
    prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

    model = Prophet()
    model.fit(prophet_df)

    future = model.make_future_dataframe(periods=10, freq='MS')
    forecast = model.predict(future)

    future_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)
    future_forecast['Mapped_GCK'] = product_id

    forecast_results.append(future_forecast)

forecast_results_df = pd.concat(forecast_results, ignore_index=True)

plt.figure(figsize=(12,6))
for product_id in merged_data_2["Mapped_GCK"].unique():
    product_data = merged_data_2[merged_data_2["Mapped_GCK"] == product_id]
    plt.plot(product_data['YearMonth'], product_data['Sales_EUR_Log'], label=f"Product {product_id} Historical Sales", alpha=0.7)

plt.plot(forecast_results_df['ds'], forecast_results_df['yhat'], label="Forecast", linestyle='dashed', color='red')
plt.axvline(x='2022-05-01', color='black', linestyle='--', label="Forecast Start")
plt.fill_between(forecast_results_df['ds'], forecast_results_df['yhat_lower'], forecast_results_df['yhat_upper'], color='red', alpha=0.2)
plt.legend()
plt.title("Prophet Forecast for Sales")
plt.xlabel("Date")
plt.ylabel("Log Sales")
plt.show()

# XGB

In [ ]:
print(test_data.columns)
test_data.loc[:, "YearMonth"] = pd.to_datetime(test_data["Year"].astype(str) + "-" + test_data["Month"].astype(str) + "-01")

In [ ]:
# Define best parameters manually
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=9,
    subsample=1.0,
    colsample_bytree=0.6,
    random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb_log = xgb_model.predict(X_test)  # Log scale predictions
y_pred_xgb = np.expm1(y_pred_xgb_log)  # Convert back to EUR scale
y_test_actual = np.expm1(y_test)

# Compute RMSE
rmse_xgb = mean_squared_error(y_test_actual, y_pred_xgb, squared=False)
print("XGBoost RMSE:", rmse_xgb)

# **Predict for Next 10 Months**
future_X = X_test.iloc[-10:].copy()  # Use last known values
future_X = future_X.drop(columns=['YearMonth'], errors='ignore')  # Remove non-numeric columns
future_predictions_log = xgb_model.predict(future_X)
future_predictions = np.expm1(future_predictions_log)

# Generate future dates
last_date = pd.to_datetime(test_data["Year"].astype(str) + "-" + test_data["Month"].astype(str) + "-01").max()
future_dates = pd.date_range(start=last_date, periods=11, freq="MS")[1:]  # First date is excluded

# **Plot Forecast**
plt.figure(figsize=(12, 6))
sns.lineplot(x=test_data["Year"].astype(str) + "-" + test_data["Month"].astype(str), 
             y=np.expm1(test_data["Sales_EUR_Log"]), label="Actual Sales", marker="o", color="orange")
sns.lineplot(x=future_dates.strftime('%Y-%m'), y=future_predictions, label="XGBoost Forecast", linestyle="dashed", color="blue")

plt.xticks(rotation=45)
plt.xlabel("Year-Month")
plt.ylabel("Sales (EUR)")
plt.title("XGBoost Sales Forecast for the Next 10 Months")
plt.legend()
plt.grid(True)
plt.show()

# RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train Random Forest model with optimal parameters
rf_model = RandomForestRegressor(
    n_estimators=300,  # Optimal number of trees
    max_depth=20,  # Limit tree depth for better generalization
    min_samples_split=5,  # Avoid overfitting
    min_samples_leaf=2,  # More robust trees
    max_features="sqrt",  # Feature selection strategy
    random_state=42
)

rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf_log = rf_model.predict(X_test)  # Log scale predictions
y_pred_rf = np.expm1(y_pred_rf_log)  # Convert back to EUR scale
y_test_actual = np.expm1(y_test)

# Compute RMSE
rmse_rf = mean_squared_error(y_test_actual, y_pred_rf, squared=False)
print("Random Forest RMSE:", rmse_rf)

# **Predict for Next 10 Months**
future_X = X_test.iloc[-10:].copy()  # Use last known values
future_X = future_X.drop(columns=['YearMonth'], errors='ignore')  # Remove non-numeric columns
future_predictions_rf_log = rf_model.predict(future_X)
future_predictions_rf = np.expm1(future_predictions_rf_log)

# Generate future dates
last_date = pd.to_datetime(test_data["Year"].astype(str) + "-" + test_data["Month"].astype(str) + "-01").max()
future_dates = pd.date_range(start=last_date, periods=11, freq="MS")[1:]  # First date is excluded

# **Plot Forecast**
plt.figure(figsize=(12, 6))
sns.lineplot(x=test_data["Year"].astype(str) + "-" + test_data["Month"].astype(str), 
             y=np.expm1(test_data["Sales_EUR_Log"]), label="Actual Sales", marker="o", color="orange")
sns.lineplot(x=future_dates.strftime('%Y-%m'), y=future_predictions_rf, label="RF Forecast", linestyle="dotted", color="green")

plt.xticks(rotation=45)
plt.xlabel("Year-Month")
plt.ylabel("Sales (EUR)")
plt.title("Random Forest Sales Forecast for the Next 10 Months")
plt.legend()
plt.grid(True)
plt.show()

# SARIMA

In [ ]:
# Prepare time-series data
train_data["YearMonth"] = pd.to_datetime(train_data["Year"].astype(str) + "-" + train_data["Month"].astype(str) + "-01")
test_data["YearMonth"] = pd.to_datetime(test_data["Year"].astype(str) + "-" + test_data["Month"].astype(str) + "-01")

# Set YearMonth as index
train_ts = train_data.set_index("YearMonth")["Sales_EUR_Log"]
test_ts = test_data.set_index("YearMonth")["Sales_EUR_Log"]

# Fit SARIMA Model
sarima_model = SARIMAX(train_ts, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
sarima_result = sarima_model.fit(disp=False)

# Make predictions
sarima_pred_log = sarima_result.predict(start=test_ts.index[0], end=test_ts.index[-1])
sarima_pred = np.expm1(sarima_pred_log)  # Convert back to original scale
test_actual = np.expm1(test_ts)

# Compute RMSE
rmse_sarima = mean_squared_error(test_actual, sarima_pred, squared=False)
print("SARIMA RMSE:", rmse_sarima)

# **Forecast for Next 10 Months**
future_dates = pd.date_range(start=test_ts.index.max(), periods=11, freq="MS")[1:]
sarima_forecast_log = sarima_result.forecast(steps=10)
sarima_forecast = np.expm1(sarima_forecast_log)  # Convert back to EUR

# **Plot Forecast**
plt.figure(figsize=(12, 6))
sns.lineplot(x=test_ts.index, y=test_actual, label="Actual Sales", marker="o", color="orange")
sns.lineplot(x=future_dates, y=sarima_forecast, label="SARIMA Forecast", linestyle="dashed", color="purple")

plt.xticks(rotation=45)
plt.xlabel("Year-Month")
plt.ylabel("Sales (EUR)")
plt.title("SARIMA Sales Forecast for the Next 10 Months")
plt.legend()
plt.grid(True)
plt.show()